# <div align="center" style="color: #ff5733;">New KPI Report</div>

## *Requestor is Jennyfer & Anne*

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
# path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

c:\Users\DwaipayanChakroborti\Myenv\reporting\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


# FPD30 January 2025 Month End

In [28]:
sq = """SELECT (case when t1.new_loan_type = 'Flex-up' and reloan_flag = 1 then 'Reloan'
else t1.new_loan_type end) new_loan_type,
case when sum(t1.obsFPD30) = 0 then 0 else ROUND(SUM(CASE when t1.defFPD30 = 1 then 1 ELSE 0 end )/sum(t1.obsFPD30)*100,3) end as percentage_of_FPD30
from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
inner JOIN prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core t2
ON
t1.loanAccountNumber = t2.loanAccountNumber
WHERE t1.flagDisbursement = 1
and 
(case when t1.new_loan_type = 'Flex-up' and reloan_flag = 1 then 'Reloan'
else t1.new_loan_type end) 
in ('Quick','Flex','SIL-Instore', 'SIL ZERO', 'SIL Competitor', 'SIL Repeat', 'Flex-up', 'Reloan')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2025-01-31')
GROUP BY 1;"""

jan25fpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
jan25fpd30

Job ID 852aa17e-caab-41b1-ae08-28e50193f7f4 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,percentage_of_FPD30
0,Flex,0.351
1,SIL Competitor,5.712
2,Flex-up,3.260
3,SIL Repeat,0.000
4,SIL-Instore,8.442
5,SIL ZERO,7.002
6,Quick,15.032
7,Reloan,3.221


# 30+, 60+, 90+ for January 2025

In [29]:
sq = """
with b as
(select 
(case when t1.new_loan_type = 'Flex-up' and reloan_flag = 1 then 'Reloan'
else t1.new_loan_type end) new_loan_type
, count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and (case when t1.new_loan_type = 'Flex-up' and reloan_flag = 1 then 'Reloan'
else t1.new_loan_type end)  in ('Quick','Flex','SIL-Instore', 'SIL ZERO', 'SIL Competitor', 'SIL Repeat', 'Reloan')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2025-01-31')
group by 1
)
select 
new_loan_type,
case when sum(obsthirtyplus) = 0 then 0 else round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) end thirtyplus,
case when sum(obsthirtyplus) = 0 then 0 else round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) end sixtyplus,
case when sum(obsnintyplus) = 0 then 0 else round(sum(nintyplus)/sum(obsnintyplus)*100, 3) end nintyplus,
from b
group by 1
order by 1;
"""

secondpartJan25 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
secondpartJan25

Job ID 5f8f8703-4264-4fa8-9c29-0cf0e5ffa71f successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,thirtyplus,sixtyplus,nintyplus
0,Flex,40.526,34.737,30.351
1,Quick,32.592,25.141,31.783
2,Reloan,20.632,16.834,18.742
3,SIL Competitor,5.796,0.000,0.000
4,SIL Repeat,0.000,0.000,0.000
5,SIL ZERO,12.655,8.512,9.745
6,SIL-Instore,28.759,23.559,26.745


# 30+, 60+, 90+ for January 2025 (without Product)

In [30]:
sq = """
with b as
(select 
count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and (case when t1.new_loan_type = 'Flex-up' and reloan_flag = 1 then 'Reloan'
else t1.new_loan_type end)  in ('Quick','Flex','SIL-Instore', 'SIL ZERO', 'SIL Competitor', 'SIL Repeat', 'Reloan')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2025-01-31')
)
select 
case when sum(obsthirtyplus) = 0 then 0 else round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) end as thirtyplus,
case when sum(obsthirtyplus) = 0 then 0 else round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) end as sixtyplus,
case when sum(obsnintyplus) = 0 then 0 else round(sum(nintyplus)/sum(obsnintyplus)*100, 3) end as nintyplus,
from b
order by 1
;"""

thirdpartJan25 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
thirdpartJan25

Job ID 443cb793-deca-4f3f-8ee7-8234754c4f9f successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,thirtyplus,sixtyplus,nintyplus
0,27.959,22.496,26.581


In [31]:
jan25fpd30.head()

,new_loan_type,percentage_of_FPD30
0,Flex,0.351
1,SIL Competitor,5.712
2,Flex-up,3.260
3,SIL Repeat,0.000
4,SIL-Instore,8.442


In [32]:
secondpartJan25

,new_loan_type,thirtyplus,sixtyplus,nintyplus
0,Flex,40.526,34.737,30.351
1,Quick,32.592,25.141,31.783
2,Reloan,20.632,16.834,18.742
3,SIL Competitor,5.796,0.000,0.000
4,SIL Repeat,0.000,0.000,0.000
5,SIL ZERO,12.655,8.512,9.745
6,SIL-Instore,28.759,23.559,26.745


In [33]:
dfJan25 = jan25fpd30.merge(secondpartJan25, on='new_loan_type', how='left')
dfJan25

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,Flex,0.351,40.526,34.737,30.351
1,SIL Competitor,5.712,5.796,0.000,0.000
2,Flex-up,3.260,NaN,NaN,NaN
3,SIL Repeat,0.000,0.000,0.000,0.000
4,SIL-Instore,8.442,28.759,23.559,26.745
5,SIL ZERO,7.002,12.655,8.512,9.745
6,Quick,15.032,32.592,25.141,31.783
7,Reloan,3.221,20.632,16.834,18.742


In [34]:
thirdpartJan25.head()

,thirtyplus,sixtyplus,nintyplus
0,27.959,22.496,26.581


In [35]:
resultJan25 = pd.concat([dfJan25, thirdpartJan25], ignore_index=True)


In [36]:
resultJan25['new_loan_type'] = resultJan25['new_loan_type'].fillna("Portfolio")
resultJan25

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,Flex,0.351,40.526,34.737,30.351
1,SIL Competitor,5.712,5.796,0.000,0.000
2,Flex-up,3.260,NaN,NaN,NaN
3,SIL Repeat,0.000,0.000,0.000,0.000
4,SIL-Instore,8.442,28.759,23.559,26.745
5,SIL ZERO,7.002,12.655,8.512,9.745
6,Quick,15.032,32.592,25.141,31.783
7,Reloan,3.221,20.632,16.834,18.742
8,Portfolio,NaN,27.959,22.496,26.581


In [37]:
import pandas as pd
from openpyxl import load_workbook


In [38]:
# Load the existing Excel file
existing_excel_file = r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Anne\Jennifer Zaldivar\KRI_New_Report\KRI_Report_New\Data\20250104_KRI_Report_New.xlsx'
xls = pd.ExcelFile(existing_excel_file)

# Read each existing sheet into separate dataframes
df_january24 = pd.read_excel(xls, sheet_name='January2024')
df_february24 = pd.read_excel(xls, sheet_name='February2024')
df_march24 = pd.read_excel(xls, sheet_name='March2024')
df_april24 = pd.read_excel(xls, sheet_name='April2024')
df_may24 = pd.read_excel(xls, sheet_name='May2024')
df_june24 = pd.read_excel(xls, sheet_name='June2024')
df_july24 = pd.read_excel(xls, sheet_name='July2024')
df_aug24 = pd.read_excel(xls, sheet_name='August2024')
df_sep24 = pd.read_excel(xls, sheet_name='September2024')
df_oct24 = pd.read_excel(xls, sheet_name='October2024')
df_nov24 = pd.read_excel(xls, sheet_name='November2024')
df_dec24 = pd.read_excel(xls, sheet_name='December2024')



In [39]:
df_dec24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,A. Flex,0.313,34.742,29.421,24.570
1,B. Quick,14.192,31.646,22.617,29.956
2,C. SIL-Instore,7.858,26.568,20.447,23.323
3,D. SIL ZERO,7.251,12.354,6.254,11.726
4,E. SIL Repeat,0.000,0.000,0.000,0.000
5,F. SIL Competitor,5.983,0.000,0.000,0.000
6,G. Portfolio,NaN,26.864,20.309,24.112


In [40]:
# Mapping function to modify 'new_loan_type' values
def modify_loan_type(loan_type):
    mapping = {'Flex': 'A. Flex', 'SIL-Instore': 'C. SIL-Instore', 'Quick': 'B. Quick', 'SIL ZERO': 'D. SIL ZERO', 'SIL Repeat':'E. SIL Repeat', 'SIL Competitor':'F. SIL Competitor', 'Portfolio': 'G. Portfolio'}
    return mapping.get(loan_type, loan_type)  # Return original value if not found

# Apply the mapping function to the 'new_loan_type' column
resultJan25['new_loan_type'] = resultJan25['new_loan_type'].apply(modify_loan_type)
resultJan25.sort_values(by='new_loan_type')

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,A. Flex,0.351,40.526,34.737,30.351
6,B. Quick,15.032,32.592,25.141,31.783
4,C. SIL-Instore,8.442,28.759,23.559,26.745
5,D. SIL ZERO,7.002,12.655,8.512,9.745
3,E. SIL Repeat,0.000,0.000,0.000,0.000
1,F. SIL Competitor,5.712,5.796,0.000,0.000
2,Flex-up,3.260,NaN,NaN,NaN
8,G. Portfolio,NaN,27.959,22.496,26.581
7,Reloan,3.221,20.632,16.834,18.742


In [41]:
# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Anne\Jennifer Zaldivar\KRI_New_Report\KRI_Report_New\Data\20250204_KRI_Report_New.xlsx', engine='xlsxwriter')

# Sort each DataFrame by the first column and write to a separate worksheet, ignoring the index
df_january24.sort_values(by=df_january24.columns[0]).to_excel(writer, sheet_name='January2024', index=False)
df_february24.sort_values(by=df_february24.columns[0]).to_excel(writer, sheet_name='February2024', index=False)
df_march24.sort_values(by=df_march24.columns[0]).to_excel(writer, sheet_name='March2024', index=False)
df_april24.sort_values(by=df_april24.columns[0]).to_excel(writer, sheet_name='April2024', index=False)
df_may24.sort_values(by=df_may24.columns[0]).to_excel(writer, sheet_name='May2024', index=False)
df_june24.sort_values(by=df_june24.columns[0]).to_excel(writer, sheet_name='June2024', index=False)
df_july24.sort_values(by=df_july24.columns[0]).to_excel(writer, sheet_name='July2024', index=False)
df_aug24.sort_values(by=df_aug24.columns[0]).to_excel(writer, sheet_name='August2024', index=False)
df_sep24.sort_values(by=df_sep24.columns[0]).to_excel(writer, sheet_name='September2024', index=False)
df_oct24.sort_values(by=df_oct24.columns[0]).to_excel(writer, sheet_name='October2024', index = False)
df_nov24.sort_values(by=df_nov24.columns[0]).to_excel(writer, sheet_name='November2024', index=False)
df_dec24.sort_values(by=df_dec24.columns[0]).to_excel(writer, sheet_name='December2024', index = False)
resultJan25.sort_values(by=resultJan25.columns[0]).to_excel(writer, sheet_name='January2025', index = False)


# Close the Pandas Excel writer and output the Excel file
writer.close()